In [1]:
import tensorflow as tf
from tensorflow import keras

#   패키지 설치
#   pip install tensorflow
#   pip install keras

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

# (패션 MNIST)데이터 열기&분할
- 0: 티셔츠, 1: 바지, 2: 스웨터, 3: 드레스, 4: 코트
- 5: 샌들, 6: 셔츠, 7: 스니커즈, 8: 가방, 9: 부츠


In [2]:
(xtrain_all, ytrain_all), (xtest, ytest) =\
    keras.datasets.fashion_mnist.load_data()

print( xtrain_all.shape, xtest.shape )    # (훈련/검증/테스트 입력 샘플 갯수, 너비, 높이)

(60000, 28, 28) (10000, 28, 28)


# 데이터 스케일링(정규화)

In [3]:
xtrain_all_scaled = xtrain_all / 255.0
# xtest_scaled = xtest / 255.0

# 데이터 분할(훈련 + 검증)

In [4]:
from sklearn.model_selection import train_test_split

xtrain, xval, ytrain, yval = train_test_split(
 
    xtrain_all_scaled, ytrain_all    # x: 특성, y: 타겟
    , test_size = 0.2       # 검증 데이터 비율(0~1)
#    , stratify = target      # 훈련 데이터 클래스 비율 = 검증 데이터 클래스 비율
    , random_state = 42    # 랜덤 시드
    )

print( xtrain.shape, xval.shape, xtest.shape )    # (훈련/검증/테스트 입력 샘플 갯수, 특성 개수)

(48000, 28, 28) (12000, 28, 28) (10000, 28, 28)


# 심층 신경망

In [5]:
# 모델 생성
model = keras.Sequential()

# 입력층( 2차원 특성(너비, 높이) -> 1차원 특성(너비X높이) )
model.add(
    keras.layers.Flatten(input_shape = (28, 28))
)


# 은닉층
model.add(
    keras.layers.Dense(
    100
    , activation = 'relu')    
)

# 출력층
model.add(
    keras.layers.Dense(
    10
    , activation = 'softmax'    
    )
)

# 모델 정보
model.summary()

###############
# 층 이름(종류)    출력 크기(미니 배치 샘플 개수, 뉴런 개수)    파라미터 개수 = ( (왼쪽)뉴런 개수+1 ) X (오른쪽)뉴런 개수

# 전체 파라미터 개수
# 훈련 파라미터 개수
# 훈련X 파라미터 개수
###############

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 모델 설정
adam = tf.keras.optimizers.Adam(
    learning_rate = 0.001    # 학습률
    , beta_1 = 0.9           # 모멘텀 최적화(그레디언트 지수 감소 평균 조절값)
    , beta_2 = 0.999         # RMSprop(그레디언트 제곱 지수 감소 평균 조절값)
    )

model.compile(
    loss = 'sparse_categorical_crossentropy' # 손실 함수(이진/다중 분류 + 원-핫 인코딩 )
#   loss = 'binary/categorical_crossentropy' # 손실 함수(이진/다중 분류)
    , metrics = 'accuracy'    # 성능 지표(정확도)
    , optimizer = adam
#    , optimizer = 'adam'
    )

# 모델 학습
model.fit(xtrain, ytrain, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 33s 3ms/step - loss: 0.5273 - accuracy: 0.8135
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3949 - accuracy: 0.8584
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3557 - accuracy: 0.8719
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3307 - accuracy: 0.8794
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3096 - accuracy: 0.8866


In [7]:
# 모델 평가
model.evaluate(xval, yval) # 손실 함수, 성능 지표

375/375 [==============================] - 11s 2ms/step - loss: 0.3500 - accuracy: 0.8733


[0.3500242233276367, 0.8732500076293945]

# 정확도: 인공 신경망 < 심층 신경망